In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# !conda create -n vilbert-mt python=3.6
# !conda activate vilbert-mt
!git clone --recursive https://github.com/facebookresearch/vilbert-multi-task.git
%cd vilbert-multi-task
# https://stackoverflow.com/questions/22250483/stop-pip-from-failing-on-single-package-when-installing-with-requirements-txt
!cat requirements.txt | sed -e '/^\s*#.*$/d' -e '/^\s*$/d' | xargs -n 1 pip install

In [ ]:
!apt-get -qq install -y python-prctl
# apt-get, not pip, worked for this module

In [ ]:
# https://stackoverflow.com/questions/57284345/how-to-install-nvidia-apex-on-google-colab
!git clone https://github.com/NVIDIA/apex
%cd apex
# !pip install -v --disable-pip-version-check --no-cache-dir ./
# https://github.com/NVIDIA/apex/issues/161
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
%cd ../
%pwd

In [ ]:
# TypeError: Couldn't build proto file into descriptor pool!
## https://github.com/ValvePython/csgo/issues/8
!pip uninstall -y protobuf
!pip install --no-binary=protobuf protobuf

In [ ]:
# TypeError: Conflict register for file "tensorboard/compat/proto/tensor_shape.proto": tensorboard.TensorShapeProto is already defined in file "tensorboardX/src/tensor_shape.proto".
## xx
# !pip install allennlp
# !pip install allennlp-models
!pip install tensorboardX==1.8

In [ ]:
!pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
%cd mmf
!pip install --editable .

In [ ]:
# Decrease batch size to avoid memory error
## https://datascience.stackexchange.com/questions/13669/how-to-export-one-cell-of-a-jupyter-notebook
%%writefile vilbert_tasks.yml
TASK1:
  name: VQA
  type: VL-classifier
  loss: BCEWithLogitLoss
  process: normal
  task_id: 1
  dataroot: datasets/VQA/
  features_h5path1: datasets/coco/features_100/COCO_trainval_resnext152_faster_rcnn_genome.lmdb
  features_h5path2: ''
  train_annotations_jsonpath: ''
  val_annotations_jsonpath: ''
  max_seq_length: 23
  max_region_num: 101
  batch_size: 8
  eval_batch_size: 8
  train_split: trainval
  val_split: minval
  lr: 0.00004
  num_epoch: 20
TASK2:
  name: GenomeQA
  type: VL-classifier
  loss: BCEWithLogitLoss
  process: normal
  task_id: 2
  dataroot: datasets/visual_genome/
  features_h5path1: datasets/visual_genome/vg_resnext152_faster_rcnn_genome.lmdb
  features_h5path2: ''
  train_annotations_jsonpath: ''
  val_annotations_jsonpath: ''
  max_seq_length: 26
  max_region_num: 101
  batch_size: 128
  eval_batch_size: 1024
  train_split: train
  val_split: val
  lr: 0.00004
  num_epoch: 20
TASK3:
  name: VisualDialog
  type: VL-logit
  loss: CrossEntropyLoss
  process: dialog
  task_id: 3
  dataroot: datasets/visual_dialog
  features_h5path1: datasets/coco/features_100/COCO_trainval_resnext152_faster_rcnn_genome.lmdb
  features_h5path2: ''
  train_annotations_jsonpath: datasets/visual_dialog/visdial_1.0_train.json
  val_annotations_jsonpath: datasets/visual_dialog/visdial_1.0_val.json
  max_seq_length: 16
  max_region_num: 101
  batch_size: 64
  train_split: train
  val_split: val
  lr: 0.00004
  num_epoch: 20
TASK4:
  name: Visual7w
  type: V-logit-mc
  loss: BCEWithLogitLoss
  process: normal
  task_id: 4
  dataroot: datasets/visual7w
  features_h5path1: datasets/visual7w/visual7w_resnext152_faster_rcnn_genome.lmdb
  features_h5path2: datasets/visual7w/visual7w_gt_resnext152_faster_rcnn_genome.lmdb
  train_annotations_jsonpath: ''
  val_annotations_jsonpath: ''
  max_seq_length: 20
  max_region_num: 200
  batch_size: 256
  train_split: train
  val_split: val
  lr: 0.00002
  num_epoch: 20
TASK5:
  name: VCR_Q-A
  type: VL-logit
  loss: CrossEntropyLoss
  process: expand
  task_id: 5
  dataroot: datasets/VCR
  features_h5path1: datasets/VCR/VCR_resnext152_faster_rcnn_genome.lmdb
  features_h5path2: datasets/VCR/VCR_gt_resnext152_faster_rcnn_genome.lmdb
  train_annotations_jsonpath: datasets/VCR/train.jsonl
  val_annotations_jsonpath: datasets/VCR/val.jsonl
  max_seq_length: 60
  max_region_num: 101
  batch_size: 64
  train_split: train
  val_split: val
  lr: 0.00002
  num_epoch: 10
TASK6:
  name: VCR_QA-R
  type: VL-logit
  loss: CrossEntropyLoss
  process: expand
  task_id: 6
  dataroot: datasets/VCR
  features_h5path1: datasets/VCR/VCR_resnext152_faster_rcnn_genome.lmdb
  features_h5path2: datasets/VCR/VCR_gt_resnext152_faster_rcnn_genome.lmdb
  train_annotations_jsonpath: datasets/VCR/train.jsonl
  val_annotations_jsonpath: datasets/VCR/val.jsonl
  max_seq_length: 80
  max_region_num: 101
  batch_size: 64
  train_split: train
  val_split: val
  lr: 0.00002
  num_epoch: 10
TASK7:
  name: RetrievalCOCO
  type: VL-logit
  loss: CrossEntropyLoss
  process: retrieval
  task_id: 7
  dataroot: datasets/cocoRetreival
  features_h5path1: datasets/coco/features_100/COCO_trainval_resnext152_faster_rcnn_genome.lmdb
  features_h5path2: ''
  train_annotations_jsonpath: datasets/cocoRetreival/all_data_final_train_2014.jsonline
  val_annotations_jsonpath: datasets/cocoRetreival/all_data_final_test_set0_2014.jsonline
  max_seq_length: 30
  max_region_num: 101
  batch_size: 128
  train_split: train
  val_split: val
  lr: 0.00002
  num_epoch: 20
TASK8:
  name: RetrievalFlickr30k
  type: VL-logit
  loss: CrossEntropyLoss
  process: retrieval
  task_id: 8
  dataroot: datasets/flickr30k
  features_h5path1: datasets/flickr30k/flickr30k_resnext152_faster_rcnn_genome.lmdb
  features_h5path2: ''
  train_annotations_jsonpath: datasets/flickr30k/all_data_final_train_2014.jsonline
  val_annotations_jsonpath: datasets/flickr30k/all_data_final_test_set0_2014.jsonline
  max_seq_length: 30
  max_region_num: 101
  batch_size: 128
  train_split: train
  val_split: val
  lr: 0.00002
  num_epoch: 20
TASK9:
  name: refcoco
  type: V-logit
  loss: BCEWithLogitLoss
  process: normal
  task_id: 9
  dataroot: datasets/refcoco
  features_h5path1: datasets/refcoco/refcoco_unc/refcoco_resnext152_faster_rcnn_genome.lmdb
  features_h5path2: datasets/refcoco/refcoco_unc/refcoco_gt_resnext152_faster_rcnn_genome.lmdb
  train_annotations_jsonpath: ''
  val_annotations_jsonpath: ''
  max_seq_length: 20
  max_region_num: 101
  batch_size: 256
  train_split: train
  val_split: val
  lr: 0.00002
  num_epoch: 20
TASK10:
  name: refcoco+
  type: V-logit
  loss: BCEWithLogitLoss
  process: normal
  task_id: 10
  dataroot: datasets/refcoco
  features_h5path1: datasets/refcoco/refcoco+_unc/refcoco+_resnext152_faster_rcnn_genome.lmdb
  features_h5path2: datasets/refcoco/refcoco+_unc/refcoco+_gt_resnext152_faster_rcnn_genome.lmdb
  train_annotations_jsonpath: ''
  val_annotations_jsonpath: ''
  max_seq_length: 20
  max_region_num: 101
  batch_size: 256
  eval_batch_size: 1024
  train_split: train
  val_split: val
  lr: 0.00002
  num_epoch: 20
TASK11:
  name: refcocog
  type: V-logit
  loss: BCEWithLogitLoss
  process: normal
  task_id: 11
  dataroot: datasets/refcoco
  features_h5path1: datasets/refcoco/refcocog_umd/refcocog_resnext152_faster_rcnn_genome.lmdb
  features_h5path2: datasets/refcoco/refcocog_umd/refcocog_gt_resnext152_faster_rcnn_genome.lmdb
  train_annotations_jsonpath: ''
  val_annotations_jsonpath: ''
  max_seq_length: 20
  max_region_num: 101
  batch_size: 256
  train_split: train
  val_split: val
  start_iteration: 0
  lr: 0.00002
  num_epoch: 20
TASK12:
  name: NLVR2
  type: VL-binary-classifier
  loss: BCEWithLogitLoss
  process: nlvr
  task_id: 12
  dataroot: datasets/nlvr2/
  features_h5path1: datasets/nlvr2/nlvr2_resnext152_faster_rcnn_genome.lmdb
  features_h5path2: ''
  train_annotations_jsonpath: ''
  val_annotations_jsonpath: ''
  max_seq_length: 40
  max_region_num: 101
  batch_size: 64
  eval_batch_size: 512
  train_split: train
  val_split: dev
  lr: 0.00002
  num_epoch: 20
TASK13:
  name: VisualEntailment
  type: VL-tri-classifier
  loss: BCEWithLogitLoss
  process: normal
  task_id: 13
  dataroot: datasets/visual_entailment/
  features_h5path1: datasets/flickr30k/flickr30k_resnext152_faster_rcnn_genome.lmdb
  features_h5path2: ''
  train_annotations_jsonpath: ''
  val_annotations_jsonpath: ''
  max_seq_length: 56
  max_region_num: 101
  batch_size: 256
  eval_batch_size: 1024
  train_split: train
  val_split: dev
  lr: 0.00002
  num_epoch: 20
TASK14:
  name: GuessWhat
  type: VL-tri-classifier
  loss: BCEWithLogitLoss
  process: normal
  task_id: 14
  dataroot: datasets/guesswhat/
  features_h5path1: datasets/coco/features_100/COCO_trainval_resnext152_faster_rcnn_genome.lmdb
  features_h5path2: ''
  train_annotations_jsonpath: ''
  val_annotations_jsonpath: ''
  max_seq_length: 25
  max_region_num: 101
  batch_size: 256
  eval_batch_size: 1024
  train_split: train
  val_split: valid
  lr: 0.00004
  num_epoch: 20
TASK15:
  name: GQA
  type: VL-classifier-GQA
  loss: BCEWithLogitLoss
  process: normal
  task_id: 15
  dataroot: datasets/gqa/
  features_h5path1: datasets/gqa/gqa_resnext152_faster_rcnn_genome.lmdb
  features_h5path2: ''
  train_annotations_jsonpath: ''
  val_annotations_jsonpath: ''
  max_seq_length: 26
  max_region_num: 101
  batch_size: 128
  eval_batch_size: 1024
  train_split: trainval
  val_split: minval
  lr: 0.00004
  num_epoch: 20
TASK16:
  name: Foil
  type: VL-binary-classifier
  loss: CrossEntropyLoss
  process: normal
  task_id: 16
  dataroot:  datasets/Foil
  features_h5path1: datasets/coco/features_100/COCO_trainval_resnext152_faster_rcnn_genome.lmdb
  features_h5path2: ''
  train_annotations_jsonpath:  datasets/Foil/foilv1.0_train_2017.json
  val_annotations_jsonpath:  datasets/Foil/foilv1.0_test_2017.json
  max_seq_length: 20
  max_region_num: 101
  batch_size: 256
  eval_batch_size: 1024
  train_split: train
  val_split: val
  lr: 0.00004
  num_epoch: 20
TASK17:
  name: GuessWhatPointing
  type: V-logit-mc
  loss: BCEWithLogitLoss
  process: normal
  task_id: 17
  dataroot: datasets/guesswhat/
  features_h5path1: datasets/coco/features_100/COCO_trainval_resnext152_faster_rcnn_genome.lmdb
  features_h5path2: datasets/guesswhat/guesswhat_gt_resnext152_faster_rcnn_genome.lmdb
  train_annotations_jsonpath: ''
  val_annotations_jsonpath: ''
  max_seq_length: 256
  max_region_num: 306
  batch_size: 64
  train_split: train
  val_split: valid
  lr: 0.00002
  num_epoch: 20
TASK18:
  name: FlickrGrounding
  type: V-logit
  loss: BCEWithLogitLoss
  process: normal
  task_id: 18
  dataroot: datasets/flickr30k/
  features_h5path1: datasets/flickr30k/flickr30k_resnext152_faster_rcnn_genome.lmdb
  features_h5path2: datasets/flickr30k/flickr30k_gt_resnext152_faster_rcnn_genome.lmdb
  train_annotations_jsonpath: ''
  val_annotations_jsonpath: ''
  max_seq_length: 24
  max_region_num: 200
  batch_size: 256
  train_split: train
  val_split: val
  lr: 0.000002
  num_epoch: 20

In [ ]:
!python train_tasks.py --bert_model bert-base-uncased --from_pretrained multi_task_model.bin --config_file config/bert_base_6layer_6conect.json --tasks 1 --lr_scheduler 'warmup_linear' --train_iter_gap 4 --task_specific_tokens --save_name finetune_from_multi_task_model

In [ ]:
# install all the data
!gcloud auth login
!mkdir -p /root/.cache/torch/data/datasets/twitter
!gsutil -m cp -r gs://{bucket_name}/satire/* /root/.cache/torch/data/datasets/twitter

In [ ]:
# free up some space
!rm -r /root/.cache/torch/data/datasets/twitter/defaults/images
!rm -rf /usr/local/lib/python2.7 /swift /usr/local/cuda-10.0 /tensorflow-1.15.2/ /content/sample_data/

In [ ]:
!mmf_run config=projects/twitter/configs/vilbert/from_cc.yaml model=vilbert dataset=satire